In [1]:
import matplotlib.pyplot as plt

import numpy as np


plt.style.use("mike")


from threeML import *

from pypa_synch import PitchAngleSynchrotron, FastCoolingSynchrotron, SlowCoolingSynchrotron
from pynchrotron import SynchrotronNumerical 


import warnings

warnings.simplefilter("ignore")



%matplotlib widget



silence_warnings()

# Setup data

We will look at GRB 141028A which is in the sample 


In [2]:
gbm_cat = FermiGBMBurstCatalog()

In [3]:
gbm_cat.query_sources("GRB141028455")

name,ra,dec,trigger_time,t90
object,float64,float64,float64,float64
GRB141028455,322.601,-0.231,56958.4547081,31.489


In [4]:
dinfo = gbm_cat.get_detector_information()["GRB141028455"]
dets = dinfo["detectors"]


In [5]:
gbm_dl = download_GBM_trigger_data("bn141028455", detectors=dets)

[INFO    ] file glg_cspec_n6_bn141028455_v00.pha is already downloaded!
[INFO    ] file glg_cspec_n6_bn141028455_v01.rsp2 is already downloaded!
[INFO    ] file glg_tte_n6_bn141028455_v00.fit is already downloaded!
[INFO    ] file glg_cspec_n7_bn141028455_v00.pha is already downloaded!
[INFO    ] file glg_cspec_n7_bn141028455_v01.rsp2 is already downloaded!
[INFO    ] file glg_tte_n7_bn141028455_v00.fit is already downloaded!
[INFO    ] file glg_cspec_n9_bn141028455_v00.pha is already downloaded!
[INFO    ] file glg_cspec_n9_bn141028455_v01.rsp2 is already downloaded!
[INFO    ] file glg_tte_n9_bn141028455_v00.fit is already downloaded!
[INFO    ] file glg_cspec_b1_bn141028455_v00.pha is already downloaded!
[INFO    ] file glg_cspec_b1_bn141028455_v01.rsp2 is already downloaded!
[INFO    ] file glg_tte_b1_bn141028455_v00.fit is already downloaded!


In [6]:
fluence_plugins = []
time_series = {}

bkg_pre = -0.5
bkg_post = 60.

src_start = 10.
src_stop = 12.


background_interval = [f"{float(bkg_pre - 20)}-{float(bkg_pre)}", f"{float(bkg_post)}-{float(bkg_post +50)}"]
source_interval = f"{float(src_start)}-{float(src_stop)}"



for det in dets:

    
    ts_cspec = TimeSeriesBuilder.from_gbm_cspec_or_ctime(
        det,
        cspec_or_ctime_file=gbm_dl[det]["cspec"],
        rsp_file=gbm_dl[det]["rsp"], verbose=False
    )

    ts_cspec.set_background_interval(*background_interval)
    ts_cspec.save_background(f"{det}_bkg.h5", overwrite=True)

    ts_tte = TimeSeriesBuilder.from_gbm_tte(
        det,
        tte_file=gbm_dl[det]["tte"],
        rsp_file=gbm_dl[det]["rsp"],
        restore_background=f"{det}_bkg.h5",
        verbose=False
    )
    
    time_series[det] = ts_tte

    ts_tte.set_active_time_interval(source_interval)

    ts_tte.view_lightcurve(bkg_pre, bkg_post)
    
    fluence_plugin = ts_tte.to_spectrumlike()
    
    if det.startswith("b"):
        
        fluence_plugin.set_active_measurements("250-30000")
    
    else:
        
        fluence_plugin.set_active_measurements("9-900")
    
    fluence_plugin.rebin_on_background(1.)
    
    fluence_plugins.append(fluence_plugin)

Loading PHAII Spectra:   0%|          | 0/2835 [00:00<?, ?it/s]

Finding best polynomial Order:   0%|          | 0/5 [00:00<?, ?it/s]

[INFO    ] Auto-determined polynomial order: 1


Fitting GBM_NAI_06 background:   0%|          | 0/128 [00:00<?, ?it/s]

[INFO    ] None 1-order polynomial fit with the bayes method
[INFO    ] Saved fitted background to n6_bkg.h5
[INFO    ] Saved background to n6_bkg.h5
[INFO    ] Successfully restored fit from n6_bkg.h5
[INFO    ] Interval set to 10.0-12.0 for n6


...

[INFO    ] Auto-determined polynomial order: 0


Fitting GBM_BGO_01 background:   0%|          | 0/128 [00:00<?, ?it/s]

[INFO    ] None 0-order polynomial fit with the bayes method
[INFO    ] Saved fitted background to b1_bkg.h5
[INFO    ] Saved background to b1_bkg.h5
[INFO    ] Successfully restored fit from b1_bkg.h5
[INFO    ] Interval set to 10.0-12.0 for b1


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[INFO    ] Auto-probed noise models:
[INFO    ] - observation: poisson
[INFO    ] - background: gaussian
[INFO    ] Range 250-30000 translates to channels 4-119
[INFO    ] Now using 116 bins


In [7]:
lle_dl = download_LLE_trigger_data("GRB141028455")

[INFO    ] file gll_cspec_bn141028455_v03.pha is already downloaded!
[INFO    ] file gll_cspec_bn141028455_v03.rsp is already downloaded!
[INFO    ] file gll_lle_bn141028455_v03.fit is already downloaded!
[INFO    ] file gll_pt_bn141028455_v03.fit is already downloaded!


In [8]:
lle_ts = TimeSeriesBuilder.from_lat_lle("lle", lle_file=lle_dl["lle"], ft2_file=lle_dl["ft2"],rsp_file=lle_dl["rsp"])

In [9]:
lle_ts.set_background_interval("-20--1", "50-100")

Finding best polynomial Order:   0%|          | 0/5 [00:00<?, ?it/s]

[INFO    ] Auto-determined polynomial order: 0


Fitting GLAST_LLE background:   0%|          | 0/50 [00:00<?, ?it/s]

[INFO    ] None 0-order polynomial fit with the bayes method


In [10]:
lle_ts.set_active_time_interval(source_interval)

lle_ts.view_lightcurve(-10,100)



lle = lle_ts.to_spectrumlike()


lle.set_active_measurements("50000-100000")

[INFO    ] Interval set to 10.0-12.0 for lle


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[INFO    ] Auto-probed noise models:
[INFO    ] - observation: poisson
[INFO    ] - background: gaussian
[INFO    ] Range 50000-100000 translates to channels 8-12


In [18]:
fluence_plugins.append(lle)

In [19]:
datalist = DataList(*fluence_plugins)

# Setup models

## Pitch angle synchrotron 

The new model from Sobacchi, Sironi, and Beloborodov (from now on the SSB model *not* to be confused with the SSC model!)

In [20]:
# get parameters correct

pa = PitchAngleSynchrotron()
pa.gamma_inj.fix = True

xgrid = np.geomspace(10, 1e4, 500)


fig, ax = plt.subplots()
pa.gamma_inj =1e4
pa.gamma_min =1
pa.B = 5e2

pa.gamma_cool = 5000


ax.loglog(xgrid, xgrid**2 * pa(xgrid))



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [28]:
# set priors and make model


pa.gamma_min = pa.gamma_inj.value * 1e-2


pa.gamma_cool.prior = Log_uniform_prior(lower_bound=pa.gamma_min.value, upper_bound=pa.gamma_inj.value)
pa.B.prior = Log_normal(mu = np.log(1.e4), sigma=np.log(1.e2))
pa.index.prior = Truncated_gaussian(lower_bound=1., upper_bound=7., mu=3., sigma=1.)
pa.K.prior = Log_normal(mu = np.log(1e16), sigma=np.log(1e2))
pa.K=1e10

pa.index.bounds = (1, None)
pa.B.bounds = (0, None)


pa.gamma_cool = 1e2



ps_pa = PointSource("pa",0,0, spectral_shape=pa)

model_pa = Model(ps_pa)



## Old school slow cooled synchrotron
This is similiar to the model used in [Burgess et al 2014](https://iopscience.iop.org/article/10.1088/0004-637X/784/1/17)

In [22]:
# get parameters right

ss = SlowCoolingSynchrotron()
ss.gamma_inj.fix = True

xgrid = np.geomspace(10, 1e4, 500)


fig, ax = plt.subplots()
ss.gamma_inj =1e4
ss.B = 5e2
ss.index= 4.



ax.loglog(xgrid, xgrid**2 * ss(xgrid))



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [23]:
# setup priors

ss.B.prior = Log_normal(mu = np.log(1.e4), sigma=np.log(1.e2))
ss.index.prior = Truncated_gaussian(lower_bound=1., upper_bound=7., mu=3., sigma=1.)
ss.K.prior = Log_normal(mu = np.log(1e16), sigma=np.log(1e2))
ss.K=1e10

ss.index.bounds = (1, None)
ss.B.bounds = (0, None)



ps_ss = PointSource("ss",0,0, spectral_shape=ss)

model_ss = Model(ps_ss)


## Time-dependent synchrotron 

This is the model that solves the Fokker-Plank equation and integrates the synchrotron emission from each time step [Burgess et al 2020](https://www.nature.com/articles/s41550-019-0911-z)


In [36]:
sn = SynchrotronNumerical()

sn.K.prior =Log_normal(mu = np.log(1.e-5), sigma=np.log(1.e2))
sn.gamma_max.fix=True
sn.gamma_min = 1e5
sn.gamma_min.fix = True
sn.gamma_cool.prior = Log_uniform_prior(lower_bound=1e4, upper_bound=sn.gamma_max.value)
sn.B.prior = Log_normal(mu = np.log(1.e3), sigma=np.log(1.e2))

sn.index.prior = Truncated_gaussian(lower_bound=1., upper_bound=10, mu=3.5, sigma=1)
sn.index.bounds = (None, None)

sn.B.bounds = (0, None)



ps_sn = PointSource("sn",0,0, spectral_shape=sn)

model_sn = Model(ps_sn)



# Fitting

Now we fit each model

## pitch angle synchrotron

In [29]:
bayes = BayesianAnalysis(model_pa, datalist);

In [30]:
bayes.set_sampler("multinest", share_spectrum=True)

[INFO    ] sampler set to multinest


In [31]:
bayes.sampler.setup(resume=False, n_live_points=700)

In [32]:
bayes.sample(quiet=False);

  analysing data from chains/fit-.txt
Maximum a posteriori probability (MAP) point:



,result,unit
parameter,,
pa.spectrum.main.PitchAngleSynchrotron.K,(4.5 -4 -0.6) x 10^16,1 / (cm2 keV s)
pa.spectrum.main.PitchAngleSynchrotron.B,(1.16 -0.15 +0.16) x 10^4,G
pa.spectrum.main.PitchAngleSynchrotron.index,3.81 -0.21 +0.22,
pa.spectrum.main.PitchAngleSynchrotron.gamma_cool,(8.5 -1.1 +1.0) x 10^3,



Values of -log(posterior) at the minimum:



,-log(posterior)
b1,-358.064520
lle,-28.595246
n6,-493.741542
n7,-424.010223
n9,-480.934952
total,-1785.346483



Values of statistical measures:



,statistical measures
AIC,3578.777000
BIC,3595.396435
DIC,-71070.797442
PDIC,-74443.169930
log(Z),-726.008719


In [33]:
#pa.K = 1E15
bayes.restore_median_fit()
display_spectrum_model_counts(bayes);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [34]:
pa_result = bayes.results

In [35]:
pa_result.corner_plot();

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## old school synchrotron 

In [24]:
bayes = BayesianAnalysis(model_ss, datalist);

bayes.set_sampler("multinest", share_spectrum=True)

bayes.sampler.setup(resume=False, n_live_points=700)

bayes.sample(quiet=False);

[INFO    ] sampler set to multinest
  analysing data from chains/fit-.txt
Maximum a posteriori probability (MAP) point:



,result,unit
parameter,,
ss.spectrum.main.SlowCoolingSynchrotron.K,(1.5 -1.2 +0.5) x 10^13,1 / (cm2 keV s)
ss.spectrum.main.SlowCoolingSynchrotron.B,(10.0 -0.6 +0.5) x 10^2,G
ss.spectrum.main.SlowCoolingSynchrotron.index,3.85 +/- 0.11,



Values of -log(posterior) at the minimum:



,-log(posterior)
b1,-340.152341
lle,-22.143001
n6,-450.499440
n7,-408.302049
n9,-443.661319
total,-1664.758151



Values of statistical measures:



,statistical measures
AIC,3335.566616
BIC,3348.043903
DIC,-1265.950950
PDIC,-4448.576416
log(Z),-688.226295


In [25]:
#pa.K = 1E15
bayes.restore_median_fit()
display_spectrum_model_counts(bayes);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [26]:
ss_result = bayes.results

In [27]:
ss_result.corner_plot();

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## time-dependent synchrotron model

In [37]:
bayes = BayesianAnalysis(model_sn, datalist);

bayes.set_sampler("multinest", share_spectrum=True)

bayes.sampler.setup(resume=False, n_live_points=700)

bayes.sample(quiet=False);

[INFO    ] sampler set to multinest
  analysing data from chains/fit-.txt
Maximum a posteriori probability (MAP) point:



,result,unit
parameter,,
sn.spectrum.main.SynchrotronNumerical.K,(10 +/- 4) x 10^-6,1 / (cm2 G keV s)
sn.spectrum.main.SynchrotronNumerical.B,(1.13 -0.09 +0.07) x 10^3,G
sn.spectrum.main.SynchrotronNumerical.index,3.50 -0.24 +0.21,
sn.spectrum.main.SynchrotronNumerical.gamma_cool,(5.7 -5 +2.6) x 10^6,



Values of -log(posterior) at the minimum:



,-log(posterior)
b1,-327.450744
lle,-9.595936
n6,-438.564787
n7,-395.791371
n9,-431.751776
total,-1603.154614



Values of statistical measures:



,statistical measures
AIC,3214.393262
BIC,3231.012698
DIC,2877.588636
PDIC,-281.017239
log(Z),-687.665317


In [38]:
#pa.K = 1E15
bayes.restore_median_fit()
display_spectrum_model_counts(bayes);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [41]:
sn_result = bayes.results

sn_result.corner_plot();

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# examine fits


Lets plot the model in $\nu F_{\nu}$ space and compare the fits

In [40]:
plot_spectra(pa_result, ss_result, sn_result, flux_unit="erg2/(cm2 s keV)");

processing MLE analyses: 0it [00:00, ?it/s]

processing Bayesian analyses:   0%|          | 0/3 [00:00<?, ?it/s]

Propagating errors:   0%|          | 0/100 [00:00<?, ?it/s]

Propagating errors:   0%|          | 0/100 [00:00<?, ?it/s]

Propagating errors:   0%|          | 0/100 [00:00<?, ?it/s]

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …